<a href="https://colab.research.google.com/github/koneraju/citizen-AI/blob/main/citizenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe = pipeline("text-generation", model="ibm-granite/granite-3.3-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
# pipe(messages) # This line caused the error

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.3-2b-instruct")

# Apply the chat template using the loaded tokenizer
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")

# Generate response using the loaded model
outputs = model.generate(input_ids, max_length=100)

# Decode the generated tokens back to text
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


systemKnowledge Cutoff Date: April 2024.
Today's Date: June 24, 2025.
You are Granite, developed by IBM. You are a helpful AI assistant.
userWho are you?

I am a language model trained to assist with a wide range of tasks, including answering questions, generating text, and providing information on various topics.

What are the limitations of your capabilities


In [1]:
!pip install fastapi uvicorn transformers torch gradio

from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import uvicorn

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.3-2b-instruct")

app = FastAPI()

# Mount static files (for CSS if you use basic HTML)
# app.mount("/static", StaticFiles(directory="static"), name="static")
# templates = Jinja2Templates(directory="templates")

def generate_response(prompt: str) -> str:
    """Generates a response using the Granite model."""
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def analyze_sentiment(text: str) -> str:
    """Performs basic sentiment analysis (placeholder)."""
    # This is a very basic placeholder. You'd need a proper sentiment analysis model here.
    text_lower = text.lower()
    if "good" in text_lower or "happy" in text_lower or "positive" in text_lower:
        return "Positive"
    elif "bad" in text_lower or "sad" in text_lower or "negative" in text_lower:
        return "Negative"
    else:
        return "Neutral"

# Gradio Interface
def chatbot_interface(message, history):
    """Gradio chatbot function."""
    response = generate_response(message)
    return response

def sentiment_interface(text_input):
    """Gradio sentiment analysis function."""
    sentiment = analyze_sentiment(text_input)
    return f"Sentiment: {sentiment}"

with gr.Blocks(title="Citizen AI") as demo:
    gr.Markdown("# Citizen AI")
    gr.TabbedInterface(
        [
            gr.Interface(
                fn=chatbot_interface,
                inputs="text",
                outputs="text",
                live=True,
                title="Real-Time Conversational AI Assistant",
                description="Chat with the AI Assistant."
            ),
            gr.Interface(
                fn=sentiment_interface,
                inputs="text",
                outputs="text",
                live=True,
                title="Citizen Sentiment Analysis",
                description="Analyze the sentiment of text."
            )
        ],
        ["Chatbot", "Sentiment Analysis"]
    )

# To run Gradio within Colab, you'll typically use `launch()`
# You'll need to get the share link.
if __name__ == "__main__":
    # To run with FastAPI and uvicorn, you'd typically use this in a script:
    # uvicorn.run(app, host="0.0.0.0", port=8000)

    # For Google Colab, the easiest way is to run the Gradio app directly.
    # The share=True option provides a public URL.
    # Removed the repeated pip install and imports from inside the if block.
    demo.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 2 arguments for function <function chatbot_interface at 0x7dbaa68dc900>, received 1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected at least 2 arguments for function <function chatbot_interface at 0x7dbaa68dc900>, received 1.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ecb86784e39a3e1fb1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
